<a href="https://colab.research.google.com/github/vvvu/potential-chainsaw/blob/main/pytorch-tutorial/%5BIntermediate%5D_Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper Parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.01

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.ToTensor(),
                                           download = True)

test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.ToTensor())

# Data Loader
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# Convolutional Neural Network (2 convolutional layers)
class ConvNet(nn.Module):
  def __init__(self, num_classes = 10):
    super(ConvNet, self).__init__()
    '''
    nn.Conv2d():
    in_channels: 输入数据的通道数，例如RGB图片的通道数为3，这里为黑白MNIST，通道数为1
    out_channels: 输出数据的通道数，根据Model调整
    kernel_size: 卷积核大小，可以为int，多维度卷积核则为tuple
    stride: 步长，默认为1，可以为int，多维度步长则为tuple
    padding: 零填充

    nn.BatchNorm2d():
    Batch Normalization - 批标准化，与数据的普通标准化类似，是将分散的数据统一的一种做法
    也是优化神经网络的一种方法，「具有统一规格的数据能让机器学习更容易学习到数据之中的规律」
    num_features: 输入特征的数量，因为out_channels = 16,所以这里选择为16

    nn.MaxPool2d():
    nn.MaxPool2d可以提取重要信息，去掉不重要的信息，减少计算开销
    '''
    self.layer1 = nn.Sequential( # Sequential - 按照构造函数中传递的顺序添加到模块中
        nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 5,
                  stride = 1, padding = 2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5,
                  stride = 1, padding = 2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.fc = nn.Linear(in_features = 7 * 7 * 32, out_features = num_classes)
    # fully connected layer
    # why in_features = 7 * 7 * 32?

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1) # ?
    out = self.fc(out)
    return out

model = ConvNet(num_classes).to(device)

In [6]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [7]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
            .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2597
Epoch [1/5], Step [200/600], Loss: 0.1021
Epoch [1/5], Step [300/600], Loss: 0.0352
Epoch [1/5], Step [400/600], Loss: 0.0787
Epoch [1/5], Step [500/600], Loss: 0.0169
Epoch [1/5], Step [600/600], Loss: 0.0419
Epoch [2/5], Step [100/600], Loss: 0.1031
Epoch [2/5], Step [200/600], Loss: 0.0295
Epoch [2/5], Step [300/600], Loss: 0.0538
Epoch [2/5], Step [400/600], Loss: 0.1662
Epoch [2/5], Step [500/600], Loss: 0.0266
Epoch [2/5], Step [600/600], Loss: 0.1016
Epoch [3/5], Step [100/600], Loss: 0.0187
Epoch [3/5], Step [200/600], Loss: 0.0428
Epoch [3/5], Step [300/600], Loss: 0.0092
Epoch [3/5], Step [400/600], Loss: 0.0724
Epoch [3/5], Step [500/600], Loss: 0.0098
Epoch [3/5], Step [600/600], Loss: 0.0029
Epoch [4/5], Step [100/600], Loss: 0.0336
Epoch [4/5], Step [200/600], Loss: 0.0160
Epoch [4/5], Step [300/600], Loss: 0.0178
Epoch [4/5], Step [400/600], Loss: 0.0178
Epoch [4/5], Step [500/600], Loss: 0.0306
Epoch [4/5], Step [600/600], Loss:

In [8]:
# Test the model
model.eval()
'''
eval mode
- batch norm uses moving mean/variance instead of mini-batch mean/variance

- eval() sets the module in evaluation mode. [This has any effect only on certain
modules. They will (1)make normalization layers use running statistics 
(2) deactivates Dropout layers]

- The difference between `model.eval()` and `with torch.no_grad()`
I. `model.eval()` will notify all your layers that you are in eval mode, that way,
[batchnorm or dropout] will work in eval mode instead of training mode.
II. `torch.no_grad()` impacts the autograd engine and deactivate it. It will reduce memory
usage and speed up computations but you won't be able to BP(backprop) [which you
don't want in an eval script]
'''
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test Accuracy of the model on the 10000 test images: {}%'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 99.11%
